In [1]:
import numpy as np
import tensorflow as tf
# 导入 MINST 数据集
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


W1124 22:16:38.343219 140133268801280 deprecation.py:323] From <ipython-input-1-18d432a2f05b>:5: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W1124 22:16:38.344385 140133268801280 deprecation.py:323] From /home/lx/.local/lib/python3.5/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W1124 22:16:38.345285 140133268801280 deprecation.py:323] From /home/lx/.local/lib/python3.5/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
I

Extracting MNIST_data/train-images-idx3-ubyte.gz


W1124 22:16:38.570019 140133268801280 deprecation.py:323] From /home/lx/.local/lib/python3.5/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W1124 22:16:38.572642 140133268801280 deprecation.py:323] From /home/lx/.local/lib/python3.5/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W1124 22:16:38.613996 140133268801280 deprecation.py:323] From /home/lx/.local/lib/python3.5/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and

Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
n_input = 28 # MNIST data 输入 (img shape: 28*28)
n_steps = 28 # timesteps
n_hidden = 128 # hidden layer num of features
n_classes = 10  # MNIST 列别 (0-9 ，一共10类)
batch_size = 128


tf.reset_default_graph()
# tf Graph input
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])

In [3]:
gru = tf.contrib.rnn.GRUCell(n_hidden*2)
lstm_cell = tf.contrib.rnn.LSTMCell(n_hidden)
mcell = tf.contrib.rnn.MultiRNNCell([lstm_cell,gru])


x1 = tf.unstack(x, n_steps, 1)
outputs, states = tf.contrib.rnn.static_rnn(mcell, x1, dtype=tf.float32)
pred = tf.contrib.layers.fully_connected(outputs[-1],n_classes,activation_fn = None)

W1124 22:17:04.158390 140133268801280 deprecation.py:323] From <ipython-input-3-d775d39c7c9c>:1: GRUCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
W1124 22:17:04.162944 140133268801280 deprecation.py:323] From <ipython-input-3-d775d39c7c9c>:2: LSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W1124 22:17:04.164723 140133268801280 deprecation.py:323] From <ipython-input-3-d775d39c7c9c>:3: MultiRNNCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be r

In [4]:
learning_rate = 0.001
# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


training_iters = 100000

display_step = 10

In [5]:
# 启动session
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # 计算批次数据的准确率
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print ("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print (" Finished!")

    # 计算准确率 for 128 mnist test images
    test_len = 100
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print ("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label}))


Iter 1280, Minibatch Loss= 1.599776, Training Accuracy= 0.46094
Iter 2560, Minibatch Loss= 1.353425, Training Accuracy= 0.53906
Iter 3840, Minibatch Loss= 1.190495, Training Accuracy= 0.60156
Iter 5120, Minibatch Loss= 1.084102, Training Accuracy= 0.63281
Iter 6400, Minibatch Loss= 0.869801, Training Accuracy= 0.75781
Iter 7680, Minibatch Loss= 0.575768, Training Accuracy= 0.79688
Iter 8960, Minibatch Loss= 0.676888, Training Accuracy= 0.81250
Iter 10240, Minibatch Loss= 0.441152, Training Accuracy= 0.86719
Iter 11520, Minibatch Loss= 0.500712, Training Accuracy= 0.82812
Iter 12800, Minibatch Loss= 0.513626, Training Accuracy= 0.81250
Iter 14080, Minibatch Loss= 0.317962, Training Accuracy= 0.90625
Iter 15360, Minibatch Loss= 0.441789, Training Accuracy= 0.86719
Iter 16640, Minibatch Loss= 0.444403, Training Accuracy= 0.86719
Iter 17920, Minibatch Loss= 0.287472, Training Accuracy= 0.89844
Iter 19200, Minibatch Loss= 0.225786, Training Accuracy= 0.93750
Iter 20480, Minibatch Loss= 0.21